# CSE 158 Fall 2023, Assignment 1

### Library Import

In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
from sklearn import linear_model
import numpy as np
import string
import random
import matplotlib.pyplot as plt
import tensorflow as tf

### Utility Functions

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [3]:
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u,g,d

In [4]:
allHours = []
for l in readJSON("train.json.gz"):
    allHours.append(l)

In [5]:
hoursTrain = allHours[:165000]
hoursValid = allHours[165000:]

In [6]:
hoursPerUser = defaultdict(list)
hoursPerItem = defaultdict(list)
for u,g,d in hoursTrain:
    r = d['hours_transformed']
    hoursPerUser[u].append((g, r))
    hoursPerItem[g].append((u, r))

In [7]:
itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)
for u,g,d in hoursTrain:
    itemsPerUser[u].append(g)
    usersPerItem[g].append(u)

In [8]:
userIDs, itemIDs = {}, {}
interactions = []

for u,g,d in allHours:
    r = d['hours_transformed']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not g in itemIDs: itemIDs[g] = len(itemIDs)
    interactions.append((u,g,r))

## Part 1: Would-Play Prediction

In [9]:
# Generate a negative set
userSet = set()
gameSet = set()
playedSet = set()

for u,g,d in allHours:
    userSet.add(u)
    gameSet.add(g)
    playedSet.add((u, g))

lUserSet = list(userSet)
lGameSet = list(gameSet)

notPlayed = set()
for u,g,d in hoursValid:
    g = random.choice(lGameSet)
    while (u,g) in playedSet or (u,g) in notPlayed:
        g = random.choice(lGameSet)
    notPlayed.add((u,g))

playedValid = set()
for u,g,r in hoursValid:
    playedValid.add((u,g))

In [12]:
gameCount = defaultdict(int)
totalPlayed = 0

for u,g,_ in hoursTrain:
    gameCount[g] += 1
    totalPlayed += 1

mostPopular = [(gameCount[x], x) for x in gameCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPlayed/1.5: break

In [11]:
threshold = 1.4844999999999906

### Bayesian Personalized Ranking Model

In [13]:
items = list(gameSet)

In [14]:
class BPRbatch(tf.keras.Model):
    def __init__(self, K, lamb):
        super(BPRbatch, self).__init__()
        # Initialize variables
        self.betaI = tf.Variable(tf.random.normal([len(gameSet)], stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userSet), K], stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(gameSet), K], stddev=0.001))
        # Regularization coefficient
        self.lamb = lamb
    
    # Prediction for a single instance
    def predict(self, u, i):
        p = self.betaI[i] + tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p
    
    # Regularizer
    def reg(self):
        return self.lamb * (tf.nn.l2_loss(self.betaI) +\
                            tf.nn.l2_loss(self.gammaU) +\
                            tf.nn.l2_loss(self.gammaI))
    
    def score(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        x_ui = beta_i + tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return x_ui
    
    def call(self, sampleU, sampleI, sampleJ):
        x_ui = self.score(sampleU, sampleI)
        x_uj = self.score(sampleU, sampleJ)
        return -tf.reduce_mean(tf.math.log(tf.math.sigmoid(x_ui - x_uj)))

In [15]:
# Leverage powerful Adam optimizer with learning rate 0.1
optimizer = tf.keras.optimizers.Adam(0.1)

In [16]:
# Initialize the model
modelBPR = BPRbatch(5, 0.00001)

In [17]:
def trainingStepBPR(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleJ = [],[],[]
        for _ in range(Nsamples):
            u,i,_ = random.choice(interactions) # positive sample
            j = random.choice(items) # negative sample
            while j in itemsPerUser[u]:
                j = random.choice(items)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleJ.append(itemIDs[j])
        
        loss = model(sampleU, sampleI, sampleJ)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for 
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [18]:
# Run 150 batches of gradient descent
for i in range(150):
    obj = trainingStepBPR(modelBPR, interactions)
    if (i % 10 == 9): print("Iteration " + str(i+1) + ", objective = " + str(obj))

Iteration 10, objective = 0.51650816
Iteration 20, objective = 0.48204634
Iteration 30, objective = 0.47219062
Iteration 40, objective = 0.46221066
Iteration 50, objective = 0.45698476
Iteration 60, objective = 0.45202968
Iteration 70, objective = 0.44692212
Iteration 80, objective = 0.4490565
Iteration 90, objective = 0.44585273
Iteration 100, objective = 0.4412405
Iteration 110, objective = 0.44074723
Iteration 120, objective = 0.44516915
Iteration 130, objective = 0.44702855
Iteration 140, objective = 0.4419574
Iteration 150, objective = 0.44199258


### Evaluation of the model

In [19]:
interactionsTrain = interactions[:165000]
interactionsTest = interactions[165000:]

In [20]:
interactionsTestPerUser = defaultdict(set)
itemSet = set()
for u,i,_ in interactionsTest:
    interactionsTestPerUser[u].add(i)
    itemSet.add(i)

In [21]:
def AUCu(u, N): # N samples per user
    win = 0
    if N > len(interactionsTestPerUser[u]):
        N = len(interactionsTestPerUser[u])
    positive = random.sample(interactionsTestPerUser[u],N)
    negative = random.sample(gameSet.difference(interactionsTestPerUser[u]),N)
    for i,j in zip(positive,negative):
        si = modelBPR.predict(userIDs[u], itemIDs[i]).numpy()
        sj = modelBPR.predict(userIDs[u], itemIDs[j]).numpy()
        if si > sj:
            win += 1
    return win/N

def AUC():
    av = []
    for u in interactionsTestPerUser:
        av.append(AUCu(u, 10))
    return sum(av) / len(av)

In [22]:
AUC()

/var/folders/8f/_6h5xprs32dfmm3jfs0g7k880000gn/T/ipykernel_10546/1187885824.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  positive = random.sample(interactionsTestPerUser[u],N)
/var/folders/8f/_6h5xprs32dfmm3jfs0g7k880000gn/T/ipykernel_10546/1187885824.py:6: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  negative = random.sample(gameSet.difference(interactionsTestPerUser[u]),N)


0.8362142940422045

### Output Predictions

In [23]:
# Get games to predict for each user
pred_user_items = defaultdict(list)
for l in open('pairs_Played.csv'):
    if l.startswith("userID"):
        continue
    u,g = l.strip().split(',')
    if u in userIDs and g in itemIDs:
        score = modelBPR.predict(userIDs[u], itemIDs[g]).numpy()
    # If game is popular, recommend anyway
    elif g in return1:
        score = 100
    # If pair is unseen and game is not popular, would not recommend
    else:
        score = -100
    pred_user_items[u].append((g, score))

In [24]:
# Sort the game scores for each user
for key in pred_user_items:
    pred_user_items[key].sort(key=lambda x:x[1])
    pred_user_items[key].reverse()

In [25]:
# Check the data structure
pred_user_items

defaultdict(list,
            {'u04836696': [('g41031307', 1.7873683),
              ('g39176429', 0.7380042),
              ('g18529610', 0.3750943),
              ('g08102513', -0.10150409),
              ('g80808634', -0.36892968),
              ('g92711177', -2.2616596)],
             'u32377855': [('g46446145', 3.401711),
              ('g81608348', 2.6432543),
              ('g34193208', 2.1670759),
              ('g22244120', 2.0923254),
              ('g02273341', 2.0328453),
              ('g85900991', 1.9151305),
              ('g56621675', 1.8948737),
              ('g95702441', 1.5647134),
              ('g55811676', 1.4309304),
              ('g57016851', 1.2396044),
              ('g22161219', 1.2320784),
              ('g10444367', 1.2245648),
              ('g23131507', 0.92046046),
              ('g12792583', 0.7485879),
              ('g16246767', 0.6728389),
              ('g78336897', 0.6101606),
              ('g70852781', 0.54679954),
              ('g02637258', 0

In [26]:
# Obtain positive and negative labels
positive_pairs = []
negative_pairs = []
for key in pred_user_items:
    for g,s in pred_user_items[key][:len(pred_user_items[key])//2]:
        positive_pairs.append((key,g))
    for g,s in pred_user_items[key][len(pred_user_items[key])//2:]:
        negative_pairs.append((key,g))

In [27]:
# Write to the output file.
predictions = open("predictions_Played.csv", 'w')
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    if (u,g) in positive_pairs:
        pred = 1
    else:
        pred = 0
    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')

predictions.close()

## Part 2: Time-Played Prediction